In [22]:
import pandas as pd
import numpy as np
from pathlib import Path

IN_CSV = Path("final_data.csv")                
OUT_DIR = Path("out")                     
OUT_DIR.mkdir(parents=True, exist_ok=True)

OUT_ACCOUNTS = OUT_DIR / "risk_accounts_min_metrics.csv"
OUT_MANAGERS = OUT_DIR / "summary_by_manager.csv"

df = pd.read_csv(IN_CSV)
df.columns = [c.strip() for c in df.columns]

num_cols = [
    "cal_health_factor_scaled",
    "cal_health_factor",
    "cal_borrowed_amt_with_interest",
    "borrowed_amt_with_interest_usd",
    "cal_total_value",
    "total_value_usd",
    "underlying_price_usd",
    "available_liquidity_underlying",
    "available_liquidity_usd",
    "base_borrow_apy",
    "leverage",
]
for c in num_cols:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")

# Производные поля
if "cal_health_factor_scaled" in df.columns:
    df["hf_raw"] = df["cal_health_factor_scaled"] / 10000.0
elif "cal_health_factor" in df.columns:
    df["hf_raw"] = df["cal_health_factor"] / 10000.0

if "leverage" in df.columns:
    df["leverage_x"] = df["leverage"] / 100.0

if "base_borrow_apy" in df.columns:
    df["borrow_rate_annual"] = df["base_borrow_apy"] / 100.0

if "borrowed_amt_with_interest_usd" not in df.columns:
    if "cal_borrowed_amt_with_interest" in df.columns and "underlying_price_usd" in df.columns:
        df["borrowed_amt_with_interest_usd"] = (
            df["cal_borrowed_amt_with_interest"] * df["underlying_price_usd"]
        )

if "available_liquidity_usd" not in df.columns:
    if "available_liquidity_underlying" in df.columns and "underlying_price_usd" in df.columns:
        df["available_liquidity_usd"] = (
            df["available_liquidity_underlying"] * df["underlying_price_usd"]
        )

df["liq_gap_usd"] = np.maximum(
    0.0,
    df.get("borrowed_amt_with_interest_usd", np.nan)
    - df.get("available_liquidity_usd", np.nan),
)
df["liq_gap_underlying"] = np.maximum(
    0.0,
    df.get("cal_borrowed_amt_with_interest", np.nan)
    - df.get("available_liquidity_underlying", np.nan),
)

if "hf_raw" in df.columns:
    df = df[df["hf_raw"] <= 1.10].copy()

account_cols = [
    "session_id",
    "account",
    "credit_manager",
    "since_timestamp",
    "cal_health_factor_scaled",
    "hf_raw",
    "cal_borrowed_amt_with_interest",
    "borrowed_amt_with_interest_usd",
    "cal_total_value",
    "total_value_usd",
    "leverage",
    "leverage_x",
    "underlying_token",
    "underlying_price_usd",
    "available_liquidity_underlying",
    "available_liquidity_usd",
    "liq_gap_underlying",
    "liq_gap_usd",
    "base_borrow_apy",
    "borrow_rate_annual",
]
account_cols = [c for c in account_cols if c in df.columns]
df_accounts = df[account_cols].sort_values(["hf_raw", "total_value_usd"], ascending=[True, False])
df_accounts.to_csv(OUT_ACCOUNTS, index=False)

# По менеджерам 
group_cols = ["credit_manager"]
agg = {"session_id": "count"}
for c in [
    "hf_raw",
    "borrowed_amt_with_interest_usd",
    "available_liquidity_usd",
    "liq_gap_usd",
    "base_borrow_apy",
    "borrow_rate_annual",
]:
    if c in df.columns:
        if c == "hf_raw":
            agg[c] = ["min", "median"]
        elif c in ("base_borrow_apy", "borrow_rate_annual"):
            agg[c] = "median"
        else:
            agg[c] = "sum"

g = df.groupby(group_cols, dropna=False).agg(agg)
g.columns = ["_".join(col) if isinstance(col, tuple) else col for col in g.columns]
g = g.reset_index()

rename_map = {
    "session_id_count": "risky_accounts",
    "hf_raw_min": "hf_min",
    "hf_raw_median": "hf_median",
    "borrowed_amt_with_interest_usd_sum": "debt_usd_sum",
    "available_liquidity_usd_sum": "available_liquidity_usd_sum",
    "liq_gap_usd_sum": "liq_gap_usd_sum",
    "base_borrow_apy_median": "base_borrow_apy_median",
    "borrow_rate_annual_median": "borrow_rate_annual_median",
}
g = g.rename(columns=rename_map)

sort_cols = [c for c in ["liq_gap_usd_sum", "debt_usd_sum"] if c in g.columns]
if sort_cols:
    g = g.sort_values(sort_cols, ascending=[False] * len(sort_cols))

g.to_csv(OUT_MANAGERS, index=False)

print(f"Файл с аккаунтами: {OUT_ACCOUNTS}")
print(f"Файл с менеджерами: {OUT_MANAGERS}")

Файл с аккаунтами: out/risk_accounts_min_metrics.csv
Файл с менеджерами: out/summary_by_manager.csv


In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from textwrap import shorten

IN_ACCOUNTS = Path("out/risk_accounts_min_metrics.csv")
IN_MANAGERS = Path("out/summary_by_manager.csv") 
TOKENS_ALIASES = Path("out/tokens_aliases.csv")   
MANAGER_ALIASES = Path("out/manager_aliases.csv")  
RESULTS = Path("results")
RESULTS.mkdir(parents=True, exist_ok=True)

df_acc = pd.read_csv(IN_ACCOUNTS)
if IN_MANAGERS.exists():
    _df_man_unused = pd.read_csv(IN_MANAGERS)

def short_addr(a: str) -> str:
    a = str(a) if pd.notna(a) else ""
    return f"{a[:6]}…{a[-4:]}" if len(a) >= 10 else a

token_map = {}
if TOKENS_ALIASES.exists():
    tdf = pd.read_csv(TOKENS_ALIASES)
    if {"address","symbol"}.issubset(tdf.columns):
        token_map = dict(zip(tdf["address"].astype(str).str.lower(), tdf["symbol"].astype(str)))

manager_map = {}
if MANAGER_ALIASES.exists():
    mdf = pd.read_csv(MANAGER_ALIASES)
    if {"credit_manager","label"}.issubset(mdf.columns):
        manager_map = dict(zip(mdf["credit_manager"].astype(str).str.lower(), mdf["label"].astype(str)))

if "underlying_token" in df_acc.columns:
    df_acc["underlying_symbol"] = df_acc["underlying_token"].astype(str).str.lower().map(token_map)
    df_acc["underlying_symbol"] = df_acc["underlying_symbol"].fillna(df_acc["underlying_token"].astype(str).map(short_addr))
else:
    df_acc["underlying_symbol"] = "UNKNOWN"

if "credit_manager" in df_acc.columns:
    df_acc["manager_label"] = df_acc["credit_manager"].astype(str).str.lower().map(manager_map)
    df_acc["manager_label"] = df_acc["manager_label"].fillna(df_acc["credit_manager"].astype(str).map(short_addr))
else:
    df_acc["manager_label"] = "UNKNOWN"

# -------------------------
# 1) Time-to-Liq (risk metric)
# -------------------------
for c in ["hf_raw","borrow_rate_annual","borrowed_amt_with_interest_usd","total_value_usd"]:
    if c in df_acc.columns:
        df_acc[c] = pd.to_numeric(df_acc[c], errors="coerce")

df_acc["time_to_liq_days"] = np.where(
    (df_acc["borrow_rate_annual"] > 0) & (df_acc["hf_raw"] > 1),
    np.log(df_acc["hf_raw"]) / df_acc["borrow_rate_annual"] * 365.0,
    np.inf
)

# --- Borrow-rate shock scenarios (TtL under higher borrow APY) ---
RATE_SHOCKS = {
    "r_p25": 1.25,   # +25%
    "r_p50": 1.50,   # +50%
    "r_x2":  2.00,   # ×2
    "r_x4":  4.00,   # ×4
}

def ttl_given_rate(df, factor):
    br = pd.to_numeric(df["borrow_rate_annual"], errors="coerce")
    hf = pd.to_numeric(df["hf_raw"], errors="coerce")
    ttl = np.where((br > 0) & (hf > 1), np.log(hf)/(br*factor)*365.0, 0.0)
    ttl = np.clip(ttl, 0, np.inf)  # отрицательные в 0
    return ttl

RATE_TTL_COLS = [] 
for tag, fac in RATE_SHOCKS.items():
    col = f"time_to_liq_days__{tag}"
    df_acc[col] = ttl_given_rate(df_acc, fac)
    RATE_TTL_COLS.append(col)

# -------------------------
# 2) Price stress tests: −5, −10, −15, −20 %
# -------------------------
SCENARIOS = [-0.05, -0.10, -0.15, -0.20]

def stress_hf(df, shock):
    new_tv = df["total_value_usd"] * (1.0 + shock)
    return new_tv / df["borrowed_amt_with_interest_usd"]

for s in SCENARIOS:
    tag = f"{int(abs(s)*100)}"
    df_acc[f"hf_stress_{tag}"] = stress_hf(df_acc, s)
    df_acc[f"liq_flag_{tag}"]  = df_acc[f"hf_stress_{tag}"] < 1.0
    df_acc[f"dar_{tag}_usd"]   = np.where(df_acc[f"liq_flag_{tag}"],
                                          df_acc["borrowed_amt_with_interest_usd"], 0.0)

# -------------------------
# 3) Manager-level stats (group BY readable manager_label)
# -------------------------
group_cols = ["manager_label"] 
agg = {
    "account": "count",
    "borrowed_amt_with_interest_usd": "sum",
    "available_liquidity_usd": "sum",
    "hf_raw": ["min", "median"],
    "time_to_liq_days": ["median"]
}
for s in SCENARIOS:
    agg[f"liq_flag_{int(abs(s)*100)}"] = "sum"
    agg[f"dar_{int(abs(s)*100)}_usd"] = "sum"

g = df_acc.groupby(group_cols, dropna=False).agg(agg)
g.columns = ["_".join(c) if isinstance(c, tuple) else c for c in g.columns]
g = g.rename(columns={
    "account_count": "risky_accounts",
    "borrowed_amt_with_interest_usd_sum": "debt_usd_sum",
    "available_liquidity_usd_sum": "available_liquidity_usd_sum",
    "hf_raw_min": "hf_min",
    "hf_raw_median": "hf_median",
    "time_to_liq_days_median": "tli_median_days",
}).reset_index()

for d in [7, 30, 90]:
    g[f"tli_le_{d}_count"] = (
        df_acc.assign(_f=(df_acc["time_to_liq_days"] <= d))
             .groupby("manager_label")["_f"].sum()
             .reindex(g["manager_label"]).fillna(0).astype(int).values
    )

for a,b,name in [("liq_flag_10_sum","liq_flag_5_sum","incr_10_vs_5"),
                 ("liq_flag_15_sum","liq_flag_10_sum","incr_15_vs_10"),
                 ("liq_flag_20_sum","liq_flag_15_sum","incr_20_vs_15")]:
    g[name] = (g.get(a,0) - g.get(b,0)) if (a in g.columns and b in g.columns) else 0

g_rate = (
    df_acc.groupby("manager_label", dropna=False)[RATE_TTL_COLS]
          .median()
          .reset_index()
)
g = g.merge(g_rate, on="manager_label", how="left")

rate_cols_order = [f"time_to_liq_days__{k}" for k in RATE_SHOCKS.keys()]
for rc in rate_cols_order[::-1]:
    if rc in g.columns:
        cols = list(g.columns)
        cols.insert(cols.index("tli_median_days")+1, cols.pop(cols.index(rc)))
        g = g[cols]

g.to_csv(RESULTS / "manager_stats.csv", index=False)

# -------------------------
# 4) Account-level stats summary (descriptive)
# -------------------------
acc_stats = []
def add_stat(name,val): acc_stats.append({"metric":name, "value":val})

tfinite = df_acc["time_to_liq_days"].replace(np.inf, np.nan).dropna()
for q in [0.1,0.25,0.5,0.75,0.9]:
    add_stat(f"tli_days_p{int(q*100)}", float(np.quantile(tfinite, q)) if len(tfinite) else np.nan)
for d in [7,30,90]:
    m = df_acc["time_to_liq_days"] <= d
    add_stat(f"tli_days_<=_{d}_count", int(m.sum()))
    add_stat(f"tli_days_<=_{d}_debt_usd", float(df_acc.loc[m, "borrowed_amt_with_interest_usd"].sum()))
for q in [0.1,0.25,0.5,0.75,0.9]:
    add_stat(f"hf_p{int(q*100)}", float(np.quantile(df_acc["hf_raw"], q)))
for q in [0.5,0.75,0.9,0.95]:
    add_stat(f"debt_usd_p{int(q*100)}", float(np.quantile(df_acc["borrowed_amt_with_interest_usd"], q)))
add_stat("debt_usd_sum", float(df_acc["borrowed_amt_with_interest_usd"].sum()))
pd.DataFrame(acc_stats).to_csv(RESULTS / "account_stats.csv", index=False)

# -------------------------
# 5) Top-10 most urgent accounts (by shortest TtL)
# -------------------------
top10 = df_acc.sort_values("time_to_liq_days", ascending=True).head(10).copy()
top10.to_csv(RESULTS / "top10_urgent_accounts.csv", index=False)

# -------------------------
# 6) Figures (English labels; log-axis for Debt where needed)
# -------------------------
plt.rcParams.update({"figure.dpi": 130})

# A) Top-10 by TtL
plt.figure(figsize=(12,6))
y = np.arange(len(top10))
vals = top10["time_to_liq_days"].values
plt.barh(y, vals)
for i,(t,hf,debt) in enumerate(zip(vals, top10["hf_raw"], top10["borrowed_amt_with_interest_usd"])):
    plt.text(t, i, f"  HF={hf:.3f}, Debt={debt/1e6:.2f}M", va="center", ha="left")
plt.yticks(y, [shorten(a, 38, placeholder="…") for a in top10["account"]])
plt.xlabel("Time to liquidation (days) — lower is worse")
plt.title("Top-10 Most Urgent Accounts (by Time-to-Liq)")
plt.tight_layout(); plt.savefig(RESULTS/"top10_time_to_liq.png"); plt.close()

# B) Liquidations under shocks
labels = ["Base"] + [f"-{int(abs(s)*100)}%" for s in SCENARIOS]
vals   = [int((df_acc["hf_raw"]<1.0).sum())] + [int(df_acc[f"liq_flag_{int(abs(s)*100)}"].sum()) for s in SCENARIOS]
plt.figure(figsize=(9,6)); x = np.arange(len(labels))
plt.bar(x, vals)
for xi,v in zip(x,vals):
    plt.text(xi, v + (max(vals)*0.02 if max(vals) else 0.5), str(v), ha="center", va="bottom")
plt.xticks(x, labels); plt.ylabel("Number of Accounts"); plt.title("Liquidations under Price Shocks")
plt.tight_layout(); plt.savefig(RESULTS/"stress_liquidations.png"); plt.close()

# C) Scatter: HF vs Debt (Debt axis log)
plt.figure(figsize=(10,7))
plt.scatter(df_acc["hf_raw"], df_acc["borrowed_amt_with_interest_usd"]/1e6, s=18, alpha=0.7)
plt.yscale("log"); plt.xlabel("Health Factor (normalized)"); plt.ylabel("Debt (USD Millions, log scale)")
plt.title("HF vs Debt (Debt axis on log scale)")
plt.tight_layout(); plt.savefig(RESULTS/"scatter_hf_debt.png"); plt.close()

# D) Scatter: Debt vs Time-to-Liq (Debt axis log)
plt.figure(figsize=(10,7))
plt.scatter(df_acc["borrowed_amt_with_interest_usd"]/1e6, df_acc["time_to_liq_days"], s=18, alpha=0.7)
plt.xscale("log"); plt.xlabel("Debt (USD Millions, log scale)"); plt.ylabel("Time to liquidation (days)")
plt.title("Debt vs Time-to-Liq")
plt.tight_layout(); plt.savefig(RESULTS/"scatter_debt_tli.png"); plt.close()

# E) Time-to-Liq distribution (linear)
finite = df_acc["time_to_liq_days"].replace(np.inf, np.nan).dropna()
plt.figure(figsize=(10,6))
plt.hist(finite, bins=100)
plt.xlabel("Time to liquidation (days)"); plt.ylabel("Number of Accounts")
plt.title("Distribution of Time-to-Liq")
plt.tight_layout(); plt.savefig(RESULTS/"time_to_liq_distribution.png"); plt.close()

# E2) TtL distribution (0–730 days, 14-day bins; Y = debt in USD millions) — FIXED
mask_0_730 = (df_acc["time_to_liq_days"] >= 0) & (df_acc["time_to_liq_days"] <= 730)
subset_0_730 = df_acc.loc[mask_0_730].copy()
edges_14d = np.arange(0, 730 + 14, 14)
cats_14d = pd.IntervalIndex.from_breaks(edges_14d, closed="left")
subset_0_730["tli_bin_14d"] = pd.cut(subset_0_730["time_to_liq_days"], bins=edges_14d, right=False, include_lowest=True)
sum_14d = subset_0_730.groupby("tli_bin_14d", observed=False).agg(
    debt_usd=("borrowed_amt_with_interest_usd","sum"),
    accounts=("account","count")
).reindex(cats_14d).reset_index(names="tli_bin_14d")
sum_14d["bin_left"] = sum_14d["tli_bin_14d"].apply(lambda iv: float(iv.left) if pd.notna(iv) else np.nan)
sum_14d["bin_width"] = 14.0
sum_14d.to_csv(RESULTS/"tli_debt_binned_14d_0_730.csv", index=False)

plt.figure(figsize=(12,6))
plt.bar(sum_14d["bin_left"], sum_14d["debt_usd"]/1e6, width=sum_14d["bin_width"], align="edge")
plt.xlabel("Time to liquidation (days, 14-day bins)")
plt.ylabel("Debt (USD Millions)")
plt.title("Distribution of Debt by Time-to-Liq (0–730 days, 14-day bins)")
plt.tight_layout(); plt.savefig(RESULTS/"time_to_liq_distribution_0_730_14d.png"); plt.close()

# F) Heatmap: managers × scenarios (Top-15 by -20% liquidations), SOFT COLORS
heat = g.copy()
for col in ["liq_flag_5_sum","liq_flag_10_sum","liq_flag_15_sum","liq_flag_20_sum"]:
    if col not in heat.columns: heat[col] = 0
heat = heat.sort_values("liq_flag_20_sum", ascending=False).head(15).copy()
M = heat[["liq_flag_5_sum","liq_flag_10_sum","liq_flag_15_sum","liq_flag_20_sum"]].to_numpy()

plt.figure(figsize=(12, 0.5 + 0.5*len(heat)))
im = plt.imshow(M, aspect="auto", cmap="Blues")
cbar = plt.colorbar(im)
cbar.set_label("Liquidations")
plt.xticks([0,1,2,3], ["-5%","-10%","-15%","-20%"])
plt.yticks(range(len(heat)), [shorten(n, 28, placeholder="…") for n in heat["manager_label"]])

for i in range(M.shape[0]):
    for j in range(M.shape[1]):
        if M[i,j] > 0:
            plt.text(j, i, int(M[i,j]), ha="center", va="center", color="black", fontsize=10)

plt.title("Managers vs Stress Scenarios (Top-15 by -20% liquidations)")
plt.tight_layout(); plt.savefig(RESULTS/"heatmap_stress.png"); plt.close()

# -------------------------
# 7) Scenario CSVs (fallen accounts) + manager increments
# -------------------------
for s in SCENARIOS:
    tag=f"{int(abs(s)*100)}"
    fallen = df_acc[df_acc[f"liq_flag_{tag}"]].copy()
    cols = ["account","manager_label","credit_manager","underlying_symbol",
            "borrowed_amt_with_interest_usd","hf_raw",f"hf_stress_{tag}","time_to_liq_days"]
    existing_cols = [c for c in cols if c in fallen.columns]
    fallen[existing_cols].to_csv(RESULTS/f"fallen_accounts_{tag}.csv", index=False)

g[["manager_label","tli_median_days",
   "liq_flag_5_sum","liq_flag_10_sum","liq_flag_15_sum","liq_flag_20_sum",
   "incr_10_vs_5","incr_15_vs_10","incr_20_vs_15"
]].to_csv(RESULTS/"manager_increments.csv", index=False)

# -------------------------
# 8) UNDERLYING-ASSET ANALYTICS (group BY readable underlying_symbol)
# -------------------------
if "underlying_symbol" in df_acc.columns:
    ua = df_acc.groupby("underlying_symbol", dropna=False).agg(
        accounts=("account","count"),
        debt_usd_sum=("borrowed_amt_with_interest_usd","sum"),
        hf_min=("hf_raw","min"),
        hf_median=("hf_raw","median"),
        tli_median_days=("time_to_liq_days", lambda x: np.median(x.replace([np.inf, -np.inf], np.nan)))
    ).reset_index()

    for d in [7,30,90]:
        ua[f"tli_le_{d}_count"] = (
            df_acc.assign(_f=(df_acc["time_to_liq_days"]<=d))
                 .groupby("underlying_symbol")["_f"].sum()
                 .reindex(ua["underlying_symbol"]).fillna(0).astype(int).values
        )

    for s in SCENARIOS:
        tag = f"{int(abs(s)*100)}"
        ua[f"liq_{tag}_count"] = (
            df_acc.groupby("underlying_symbol")[f"liq_flag_{tag}"].sum()
                 .reindex(ua["underlying_symbol"]).fillna(0).astype(int).values
        )
        ua[f"dar_{tag}_usd"] = (
            df_acc.groupby("underlying_symbol")[f"dar_{tag}_usd"].sum()
                 .reindex(ua["underlying_symbol"]).fillna(0.0).values
        )

    # --- Underlying-level: add TtL medians under rate shocks + Liq @ -10% ---
if "underlying_symbol" in df_acc.columns:

    ua_rate = (
        df_acc.groupby("underlying_symbol", dropna=False)[RATE_TTL_COLS]
              .median()
              .reset_index()
    )
    ua = ua.merge(ua_rate, on="underlying_symbol", how="left")

    if "liq_flag_10" in df_acc.columns:
        ua["liq_10_count"] = (
            df_acc.groupby("underlying_symbol")["liq_flag_10"]
                  .sum()
                  .reindex(ua["underlying_symbol"])
                  .fillna(0)
                  .astype(int)
                  .values
        )
    ua.to_csv(RESULTS/"underlying_stats.csv", index=False)

    # Top-10 underlyings by debt
    ua_top = ua.sort_values("debt_usd_sum", ascending=False).head(10)
    plt.figure(figsize=(12,6))
    y = np.arange(len(ua_top))
    plt.barh(y, ua_top["debt_usd_sum"]/1e6)
    for i,v in enumerate(ua_top["debt_usd_sum"]/1e6):
        plt.text(v, i, f"  {v:.1f}M", va="center", ha="left")
    plt.yticks(y, ua_top["underlying_symbol"])
    plt.xlabel("Debt (USD Millions)"); plt.title("Top-10 Underlyings by Debt")
    plt.tight_layout(); plt.savefig(RESULTS/"underlying_debt_top10.png"); plt.close()

    # Stress DAR by scenario (stacked bars for softness)
    xs = np.arange(len(ua_top))
    plt.figure(figsize=(12,6))
    base = np.zeros(len(ua_top))
    for tag, color in zip(["5","10","15","20"], ["#c6dbef","#9ecae1","#6baed6","#4292c6"]):  # мягкая сине-голубая гамма
        vals = ua_top[f"dar_{tag}_usd"]/1e6
        plt.bar(xs, vals, bottom=base, label=f"-{tag}%", color=color)
        base += vals
    for i, v in enumerate(base):
        if v > 0:
            plt.text(xs[i], v, f"{v:.1f}M", ha="center", va="bottom", fontsize=8)
    plt.xticks(xs, ua_top["underlying_symbol"])
    plt.ylabel("Debt-at-Risk (USD Millions)"); plt.title("Debt-at-Risk by Underlying (Scenarios)")
    plt.legend()
    plt.tight_layout(); plt.savefig(RESULTS/"underlying_dar_scenarios_top10.png"); plt.close()

print(f"✅ Done. All outputs saved in: {RESULTS.resolve()}")

✅ Done. All outputs saved in: /Users/phlxndr/Desktop/GearBox/September/results


In [24]:
# -------------------------
# Heatmap: Underlyings × Stress Scenarios (Liquidations, Top-10 by -20% liqs)
# -------------------------
if "underlying_symbol" in df_acc.columns:
    ua_liq = df_acc.groupby("underlying_symbol").agg(
        liq_5=("liq_flag_5","sum"),
        liq_10=("liq_flag_10","sum"),
        liq_15=("liq_flag_15","sum"),
        liq_20=("liq_flag_20","sum")
    ).reset_index()

    ua_liq = ua_liq.sort_values("liq_20", ascending=False).head(10)
    H = ua_liq[["liq_5","liq_10","liq_15","liq_20"]].to_numpy()

    plt.figure(figsize=(10, 0.5 + 0.5*len(ua_liq)))
    im = plt.imshow(H, aspect="auto", cmap="Blues")
    plt.colorbar(im, label="Liquidations")
    plt.xticks(range(H.shape[1]), ["-5%","-10%","-15%","-20%"])
    plt.yticks(range(len(ua_liq)), ua_liq["underlying_symbol"])
    for i in range(H.shape[0]):
        for j in range(H.shape[1]):
            if H[i,j] > 0:
                plt.text(j, i, int(H[i,j]), ha="center", va="center", color="black", fontsize=10)
    plt.title("Underlyings vs Stress Scenarios (Top-10 by -20% liquidations)")
    plt.tight_layout(); plt.savefig(RESULTS/"underlying_heatmap_scenarios.png"); plt.close()

# -------------------------
# Risk curve: cumulative debt vs TtL threshold (0–90 days)
#  — базовая кривая + 4 кривые под шоками ставки (разные цвета)
#  — x-ось в днях (стэп-график по каждому дню)
#  — точки/подписи только раз в 15 дней: 15,30,45,60,75,90
# -------------------------
def cum_at_thresholds(ttl_series, debt_series, thresholds):
    ttl = pd.to_numeric(ttl_series, errors="coerce").to_numpy()
    debt = pd.to_numeric(debt_series, errors="coerce").to_numpy()
    mask = np.isfinite(ttl) & np.isfinite(debt) & (ttl >= 0)
    ttl = ttl[mask]; debt = debt[mask]
    out_debt = []
    out_n = []
    for t in thresholds:
        m = ttl <= t
        out_debt.append(debt[m].sum())
        out_n.append(int(m.sum()))
    return np.array(out_debt), np.array(out_n)

CURVES = [("Base", "time_to_liq_days", "#1f77b4")] + [
    ("Rate +25%", "time_to_liq_days__r_p25", "#2ca02c"),
    ("Rate +50%", "time_to_liq_days__r_p50", "#ff7f0e"),
    ("Rate ×2",   "time_to_liq_days__r_x2",  "#d62728"),
    ("Rate ×4",   "time_to_liq_days__r_x4",  "#9467bd"),
]

thr_full = np.arange(0, 91, 1)    
thr_mark = np.arange(0, 91, 10)   

plt.figure(figsize=(10,6))
for label, col, color in CURVES:
    if col not in df_acc.columns:
        continue

    y_full, _ = cum_at_thresholds(df_acc[col], df_acc["borrowed_amt_with_interest_usd"], thr_full)
    plt.step(thr_full, y_full/1e6, where="post", label=label, color=color, linewidth=2)

    y_mark, n_mark = cum_at_thresholds(df_acc[col], df_acc["borrowed_amt_with_interest_usd"], thr_mark)
    plt.scatter(thr_mark[1:], (y_mark/1e6)[1:], s=22, color=color, zorder=3)
    for t, y, n in zip(thr_mark[1:], (y_mark/1e6)[1:], n_mark[1:]):
        plt.text(t, y, f"{y:.1f}M ({n})", ha="left", va="bottom", fontsize=8, color=color)

plt.xlabel("TtL threshold (days)")
plt.ylabel("Cumulative Debt (USD Millions)")
plt.title("Cumulative Debt at Risk vs Time-to-Liq Threshold (0–90 days)")
plt.legend()
plt.tight_layout(); plt.savefig(RESULTS/"risk_curve_cum_debt_0_90.png"); plt.close()

In [21]:
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime

# ------------------------------------------------------------
# Paths
# ------------------------------------------------------------
RESULTS = Path("results")
RESULTS.mkdir(exist_ok=True, parents=True)

OUT_HTML = RESULTS / "risk_onepager_v2.html"

# Data produced by pipeline
MANAGER_STATS     = RESULTS / "manager_stats.csv"
UNDERLYING_STATS  = RESULTS / "underlying_stats.csv"
ACCOUNT_STATS     = RESULTS / "account_stats.csv"

# Figures (we will place them section-wise, ≤2 per row)
FIGS = {  # logical sections → list of filenames
    "Stress tests": [
        "stress_liquidations.png",
        "risk_curve_cum_debt_0_90.png",
        "underlying_dar_scenarios_top10.png",
    ],
    "Heatmaps": [
        "heatmap_stress.png",
        "underlying_heatmap_scenarios.png",
    ],
    "Managers & Underlyings": [
        "top10_time_to_liq.png",
        "underlying_debt_top10.png",
    ],
    "Distributions & Scatters": [
        "time_to_liq_distribution.png",
        "time_to_liq_distribution_0_730_14d.png",
        "scatter_hf_debt.png",
        "scatter_debt_tli.png",
    ],
}

# Only keep existing images
for sec, files in list(FIGS.items()):
    FIGS[sec] = [RESULTS/f for f in files if (RESULTS/f).exists()]
# Remove empty sections
FIGS = {sec:paths for sec,paths in FIGS.items() if paths}

# ------------------------------------------------------------
# Load data (robust)
# ------------------------------------------------------------
def _safe_csv(p):
    try:
        return pd.read_csv(p)
    except Exception:
        return pd.DataFrame()

mgr = _safe_csv(MANAGER_STATS)
ua  = _safe_csv(UNDERLYING_STATS)
acc = _safe_csv(ACCOUNT_STATS)

# ------------------------------------------------------------
# Helpers
# ------------------------------------------------------------
def usd(x):
    try: return f"${x:,.0f}"
    except: return str(x)

def usd_m(x):
    try: return f"${x/1e6:,.1f}M"
    except: return str(x)

generated = datetime.utcnow().strftime("%Y-%m-%d %H:%M UTC")

# ------------------------------------------------------------
# Build richer tables
# ------------------------------------------------------------
# Manager table: Top 10 by Liq @ -20% (fallback: by Debt), with TtL medians under rate shocks
rate_cols_mgr = [c for c in ["time_to_liq_days__r_p25","time_to_liq_days__r_p50","time_to_liq_days__r_x2","time_to_liq_days__r_x4"] if c in mgr.columns]
base_cols_mgr = ["manager_label","risky_accounts","tli_median_days","liq_flag_10_sum","liq_flag_20_sum","debt_usd_sum"]
cols_mgr = base_cols_mgr + rate_cols_mgr
mgr_view = pd.DataFrame()
if not mgr.empty:
    sort_key = "liq_flag_20_sum" if "liq_flag_20_sum" in mgr.columns else ("debt_usd_sum" if "debt_usd_sum" in mgr.columns else mgr.columns[0])
    mgr_view = mgr.sort_values(sort_key, ascending=False)[cols_mgr].head(10).copy()

def format_mgr_table(df: pd.DataFrame) -> pd.DataFrame:
    if df.empty: return df
    out = df.copy()
    # formats
    if "debt_usd_sum" in out.columns:
        out["Debt (USD)"] = out["debt_usd_sum"].apply(usd_m); out.drop(columns=["debt_usd_sum"], inplace=True)
    if "tli_median_days" in out.columns:
        out["Median TtL (d)"] = out["tli_median_days"].map(lambda v: f"{float(v):.1f}" if pd.notna(v) else ""); out.drop(columns=["tli_median_days"], inplace=True)
    # rate shock medians (rename)
    rename_rate = {
        "time_to_liq_days__r_p25": "TtL @ Rate +25% (d)",
        "time_to_liq_days__r_p50": "TtL @ Rate +50% (d)",
        "time_to_liq_days__r_x2":  "TtL @ Rate ×2 (d)",
        "time_to_liq_days__r_x4":  "TtL @ Rate ×4 (d)",
    }
    for c in rate_cols_mgr:
        out[rename_rate[c]] = out[c].map(lambda v: f"{float(v):.1f}" if pd.notna(v) else "")
        out.drop(columns=[c], inplace=True)
    out = out.rename(columns={
        "manager_label":"Manager",
        "risky_accounts":"Accounts",
        "liq_flag_10_sum":"Liq @ -10%",
        "liq_flag_20_sum":"Liq @ -20%",
    })
    # desired order
    preferred = ["Manager","Accounts","Median TtL (d)","TtL @ Rate +25% (d)","TtL @ Rate +50% (d)","TtL @ Rate ×2 (d)","TtL @ Rate ×4 (d)","Liq @ -10%","Liq @ -20%","Debt (USD)"]
    out = out[[c for c in preferred if c in out.columns]]
    return out

mgr_tbl = format_mgr_table(mgr_view)

# Underlying table: Top 10 by Debt, add Liq @ -10% and TtL medians under rate shocks
rate_cols_ua = [c for c in ["time_to_liq_days__r_p25","time_to_liq_days__r_p50","time_to_liq_days__r_x2","time_to_liq_days__r_x4"] if c in ua.columns]
base_cols_ua = ["underlying_symbol","accounts","debt_usd_sum","liq_10_count","liq_20_count"]
cols_ua = [c for c in base_cols_ua + rate_cols_ua if c in ua.columns]
ua_view = pd.DataFrame()
if not ua.empty:
    sort_key = "debt_usd_sum" if "debt_usd_sum" in ua.columns else ua.columns[0]
    ua_view = ua.sort_values(sort_key, ascending=False)[cols_ua].head(10).copy()

def format_ua_table(df: pd.DataFrame) -> pd.DataFrame:
    if df.empty: return df
    out = df.copy()
    if "debt_usd_sum" in out.columns:
        out["Debt (USD)"] = out["debt_usd_sum"].apply(usd_m); out.drop(columns=["debt_usd_sum"], inplace=True)
    rename_rate = {
        "time_to_liq_days__r_p25": "TtL @ Rate +25% (d)",
        "time_to_liq_days__r_p50": "TtL @ Rate +50% (d)",
        "time_to_liq_days__r_x2":  "TtL @ Rate ×2 (d)",
        "time_to_liq_days__r_x4":  "TtL @ Rate ×4 (d)",
    }
    for c in rate_cols_ua:
        out[rename_rate[c]] = out[c].map(lambda v: f"{float(v):.1f}" if pd.notna(v) else "")
        out.drop(columns=[c], inplace=True)
    out = out.rename(columns={
        "underlying_symbol":"Underlying",
        "accounts":"Accounts",
        "liq_10_count":"Liq @ -10%",
        "liq_20_count":"Liq @ -20%",
    })
    preferred = ["Underlying","Accounts","Liq @ -10%","Liq @ -20%","TtL @ Rate +25% (d)","TtL @ Rate +50% (d)","TtL @ Rate ×2 (d)","TtL @ Rate ×4 (d)","Debt (USD)"]
    out = out[[c for c in preferred if c in out.columns]]
    return out

ua_tbl = format_ua_table(ua_view)

# KPI row (from account_stats if available; otherwise rough from mgr)
acc_kpi = _safe_csv(ACCOUNT_STATS)
def _kpi_lookup(df, key, default="—"):
    try:
        return df.loc[df["metric"]==key, "value"].iloc[0]
    except Exception:
        return default

accounts_total = len(df_acc) if not acc_kpi.empty else (int(mgr["risky_accounts"].sum()) if "risky_accounts" in mgr.columns else None)
debt_total_usd = float(_kpi_lookup(acc_kpi, "debt_usd_sum", default=np.nan)) if not acc_kpi.empty else (float(mgr["debt_usd_sum"].sum()) if "debt_usd_sum" in mgr.columns else np.nan)
hf_min     = float(_kpi_lookup(acc_kpi, "hf_p10", default=np.nan)) if False else (float(mgr["hf_min"].min()) if ("hf_min" in mgr.columns and len(mgr)) else np.nan)
hf_median  = float(_kpi_lookup(acc_kpi, "hf_p50", default=np.nan)) if not acc_kpi.empty else (float(mgr["hf_median"].median()) if ("hf_median" in mgr.columns and len(mgr)) else np.nan)
tli_median = float(_kpi_lookup(acc_kpi, "tli_days_p50", default=np.nan)) if not acc_kpi.empty else (float(mgr["tli_median_days"].median()) if ("tli_median_days" in mgr.columns and len(mgr)) else np.nan)

# ------------------------------------------------------------
# HTML (≤2 charts per row)
# ------------------------------------------------------------
def _html_table(df: pd.DataFrame, title: str) -> str:
    if df is None or df.empty:
        return f'<div class="card"><h3>{title}</h3><p><em>No data</em></p></div>'
    cols = list(df.columns)
    head = "".join([f"<th>{c}</th>" for c in cols])
    rows = []
    for _, r in df.iterrows():
        rows.append("<tr>" + "".join([f"<td>{r[c]}</td>" for c in cols]) + "</tr>")
    return f"""
    <div class="card">
      <h3>{title}</h3>
      <table>
        <thead><tr>{head}</tr></thead>
        <tbody>{''.join(rows)}</tbody>
      </table>
    </div>
    """

def _html_img(path: Path) -> str:
    return f'<div class="card"><img src="{path.name}" alt="{path.name}"/></div>'

html_sections = []
# Header + KPI
html_header = f"""
<h1>Low-HF Risks</h1>
<div class="subtle">Generated {generated}</div>
<div class="kpi">
  <div class="pill"><b>Total Accounts</b><br>{accounts_total if accounts_total is not None else '—'}</div>
  <div class="pill"><b>Total Debt</b><br>{usd(debt_total_usd) if not np.isnan(debt_total_usd) else '—'}</div>
  <div class="pill"><b>HF min</b><br>{('—' if np.isnan(hf_min) else f'{hf_min:.3f}')}</div>
  <div class="pill"><b>HF median</b><br>{('—' if np.isnan(hf_median) else f'{hf_median:.3f}')}</div>
  <div class="pill"><b>Median TtL</b><br>{('—' if np.isnan(tli_median) else f'{tli_median:.1f} d')}</div>
</div>
"""
html_sections.append(html_header)

# Tables section
html_tables = f"""
<h2>Key Tables</h2>
<div class="grid2">
  {_html_table(mgr_tbl, "Top Managers (by -20% liquidations)") if not mgr_tbl.empty else _html_table(pd.DataFrame(), "Top Managers")}
  {_html_table(ua_tbl,  "Top Underlyings (by Debt)") if not ua_tbl.empty else _html_table(pd.DataFrame(), "Top Underlyings")}
</div>
<p class="note">Rate shock TtL columns show median time-to-liquidation if borrow APY rises by +25%, +50%, ×2, ×4.</p>
"""
html_sections.append(html_tables)

# Charts, by sections
for sec, paths in FIGS.items():
    if not paths: continue
    rows = []
    row = []
    for p in paths:
        row.append(_html_img(p))
        if len(row) == 2:
            rows.append(f"<div class='grid2'>{''.join(row)}</div>")
            row = []
    if row:
        rows.append(f"<div class='grid2'>{''.join(row)}</div>")
    html_sections.append(f"<h2>{sec}</h2>" + "\n".join(rows))

# Combine HTML
html = f"""
<html>
<head>
  <meta charset="utf-8"/>
  <title>Risk One-Pager V2</title>
  <style>
    body {{ font-family: Arial, Helvetica, sans-serif; margin: 24px; color: #111; }}
    h1 {{ margin: 0 0 8px 0; }}
    h2 {{ margin: 18px 0 8px 0; }}
    h3 {{ margin: 0 0 6px 0; font-size: 14px; }}
    .subtle {{ color: #666; font-size: 12px; }}
    .note {{ color: #666; font-size: 12px; margin-top: 6px; }}
    .kpi {{ display: flex; gap: 14px; flex-wrap: wrap; margin: 10px 0 16px 0; }}
    .pill {{ border: 1px solid #e3e3e3; border-radius: 12px; padding: 10px 12px; min-width: 170px; box-shadow: 0 1px 2px rgba(0,0,0,0.04); }}
    .grid2 {{ display: grid; grid-template-columns: repeat(2, minmax(0, 1fr)); gap: 14px; }}
    .card {{ border: 1px solid #eee; border-radius: 10px; padding: 8px; background: #fff; }}
    img {{ width: 100%; height: auto; display: block; border-radius: 6px; }}
    table {{ border-collapse: collapse; width: 100%; font-size: 13px; }}
    th, td {{ border: 1px solid #e6e6e6; padding: 6px 8px; text-align: left; }}
    th {{ background: #fafafa; }}
  </style>
</head>
<body>
  {''.join(html_sections)}
</body>
</html>
"""
OUT_HTML.write_text(html, encoding="utf-8")

print("✅ One-pager V2 generated:")
print(f"- HTML: {OUT_HTML.resolve()}")

✅ One-pager V2 generated:
- HTML: /Users/phlxndr/Desktop/GearBox/September/results/risk_onepager_v2.html


/var/folders/q6/s347jm256n93cg0ph4w1dt2r0000gn/T/ipykernel_40849/3126316886.py:83: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  generated = datetime.utcnow().strftime("%Y-%m-%d %H:%M UTC")
